# Самостоятельная работа

Давайте на примере рассмотрим, как можно построить систему рекомендаций на основе популярности.

Мы будем работать с набором данных Movie Lens. Он содержит идентификаторы для каждого фильма и пользователя, который его смотрел, а также оценку, которую пользователь поставил фильму. В датасете представлено 25 000 095 оценок фильмов от 162 541 пользователя со шкалой оценок от 0.5 до 5.0.

Из этого набора нам понадобится два файла

In [39]:
import pandas as pd
import math
import numpy as np

In [2]:
ratings = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')

In [3]:
movies = pd.read_csv('data/movie.zip')

In [4]:
df=pd.merge(ratings,movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


## Признаки данных

* userId — id пользователя;
* movieId — id фильма;
* rating — выставленный пользователем рейтинг для фильма;
* timestamp — время выставления рейтинга;
* title — название фильма;
* genres — жанры, к которым относится фильм.

1. Подсчитайте, сколько раз каждый фильм встречается в наборе данных. Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.

In [5]:
df['title'].value_counts(ascending=False).head(5)

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

2. Отлично, мы нашли самые востребованные фильмы. Однако если фильм посмотрело много людей, это ещё не значит, что он им понравился. Чтобы понять, как зритель на самом деле относится к фильму, нужны более чёткие данные. К счастью, в наборе данных `Movie Lens` есть оценки каждого из зрителей.

* Найдите средний рейтинг для каждого из фильмов.
* Найдите фильмы с наивысшим средним рейтингом.
* Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.
* Впишите только название фильма, без кавычек и без года выхода на экран.

In [6]:
grouped = df.groupby(by='title')['rating'].agg(['mean']).sort_values(by=['mean', 'title'],ascending=False)
grouped.head(1)
#Yonkers Joe

,mean
title,
Yonkers Joe (2008),5.0


3. В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

Сколько таких фильмов?

In [7]:
count50 = df['title'].value_counts(ascending=False)
count50 = pd.DataFrame(count50)
count50 = count50[count50['title']>50]
count50.shape[0]
#10472

10472

4. Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, и найдите среди них фильм с наивысшей средней оценкой. В качестве ответа запишите название этого фильма без артикля и года выхода на экран.

In [8]:
count50['title'] = count50.index
f50 = pd.merge(count50,grouped, how='left',on='title')
f50.sort_values(by='mean', ascending=False)['title'].head(1)
#Shawshank Redemption

2    Shawshank Redemption, The (1994)
Name: title, dtype: object

По сути, вы только что построили простейшую рекомендательную систему. В следующем юните вы решите более объёмную задачу, пройдя все этапы от предобработки данных до оценки качества получившейся системы рекомендаций.

# Практика

В этом модуле мы обсудили, как используются рекомендательные системы, как получают данные для них и как оценивают результаты работы РС. Мы начали знакомиться с алгоритмами построения систем рекомендаций и пока успели изучить более подробно один из них — РС на основе популярности. В этом юните мы построим систему рекомендаций, основываясь именно на этом методе. Вы сможете усовершенствовать её в следующем модуле, после того как освоите другие алгоритмы.

Для начала загрузим датасет `Articles sharing and reading from CI&T DeskDrop`, включающий в себя собранные за один год логи `DeskDrop` — платформы для внутренних коммуникаций, разработанной CI&T и ориентированной на компании, использующие Google Workspace (Google G Suite). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

**Информация в наборе данных:**

* Оригинальный URL, название и текст статьи.
* Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.
* Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.

Данные включают в себя два файла:

`shared_articles.csv`;

`users_interactions.csv`.

Начнём работать с файлом shared_articles.csv. Он содержит информацию о статьях, опубликованных на платформе DeskDrop.

**Для каждой статьи есть:**

* дата публикации (временная метка),
* исходный URL-адрес,
* заголовок,
* содержание в виде обычного текста,
* язык статьи (португальский — pt или английский — en),
* информация о пользователе, который поделился статьёй (автор).

Для временной метки существует два возможных типа событий:

* `CONTENT SHARED` — статья была опубликована на платформе и доступна для пользователей;
* `CONTENT REMOVED` — статья была удалена с платформы и недоступна для дальнейших рекомендаций.
  
Для простоты мы рассматриваем здесь только тип события `CONTENT SHARED`.

1. Отфильтруйте данные так, чтобы остались только объекты с типом события `CONTENT SHARED`. Сколько таких объектов в получившейся таблице?

In [9]:
import pandas as pd

In [10]:
shared_articles = pd.read_csv('data/shared_articles.zip')

In [11]:
shared_articles_filtered = shared_articles[shared_articles['eventType']=='CONTENT SHARED']
shared_articles_filtered.shape[0]
#3047

3047

Теперь откроем второй файл — `users_interactions.csv.`

Давайте предварительно преобразуем столбцы personId, contentId в таблицах к строкам. Это преобразование пригодится нам в дальнейшем:

```
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)
```

В колонке `eventType` описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:


* VIEW — просмотр,
* LIKE — лайк,
* COMMENT CREATED — комментарий,
* FOLLOW — подписка,
* BOOKMARK — добавление в закладки.
  
В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:
```
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
```

**Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость**

In [12]:
interactions_df= pd.read_csv('data/users_interactions.zip')

In [13]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)

articles_df = shared_articles_filtered
articles_df.contentId = articles_df.contentId.astype(str)

C:\Users\пользователь\AppData\Local\Temp\ipykernel_9068\2961181235.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles_df.contentId = articles_df.contentId.astype(str)


In [14]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

2. Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [15]:
interactions_df['weight']=interactions_df['eventType'].apply(lambda x: event_type[x])

In [16]:
interactions_df['weight'].mean().round(2)
#1.24

1.24

Ранее мы говорили, что рекомендательные системы подвержены проблеме холодного старта — в таких случаях создавать рекомендации намного сложнее.

3. Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [17]:
persons = pd.DataFrame(interactions_df[['personId','contentId']].groupby(by=['personId']).agg(['unique']))
#persons = persons[persons['contentId']>5]
#persons.shape[0] 
persons['content'] = persons['contentId']['unique']
persons['content'] = persons['content'].astype(str)
persons = persons.drop(['contentId'], axis=1)
persons['len_unique'] = persons['content'].apply(lambda x: x.count(" ")+1)
persons = persons[persons['len_unique']>=5]
persons.shape[0]
#1140

1140

4. Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

В этом задании необходимо работать с данными, полученными в задании 3.

In [18]:
persons['personId'] = persons.index
persons.reset_index(drop=True, inplace=True)
persons.columns = ['content','len_unique','personId']

In [19]:
interactions_filtered = pd.merge(interactions_df, persons, how='left', on='personId')
interactions_filtered['isna'] = interactions_filtered['len_unique'].isna()
interactions_filtered = interactions_filtered[interactions_filtered['isna']==False]

In [20]:
interactions_filtered = interactions_filtered.drop(['content','len_unique','isna'], axis=1)

In [21]:
interactions_filtered.shape[0]
#69868

69868

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

```
def smooth_user_preference(x):
    return math.log(1+x, 2)
```
5. Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод `max()`.

**В этом задании необходимо работать с данными, полученными в задании 4.**

In [22]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [23]:
interactions_grouped = interactions_filtered.groupby(by=['personId','contentId'])['timestamp','weight'].agg(['max','sum'])

C:\Users\пользователь\AppData\Local\Temp\ipykernel_9068\1343130655.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  interactions_grouped = interactions_filtered.groupby(by=['personId','contentId'])['timestamp','weight'].agg(['max','sum'])


In [24]:
interactions_grouped.columns = ['tsmax','tssum','wmax', 'wsum']
interactions_grouped = interactions_grouped.drop(['tssum','wmax'], axis=1)
interactions_grouped['wsum'] = interactions_grouped['wsum'].apply(lambda x: smooth_user_preference(x))
interactions_grouped

tsmax      wsum
personId             contentId                                 
-1007001694607905623 -5065077552540450930  1470395911  1.000000
                     -6623581327558800021  1487240080  1.000000
                     -793729620925729327   1472834892  1.000000
                     1469580151036142903   1487240062  1.000000
                     7270966256391553686   1485994342  1.584963
...                                               ...       ...
998688566268269815   -401664538366009049   1474567449  1.000000
                     3456674717452933449   1478802088  2.584963
                     6881796783400625893   1474567675  1.000000
                     7174452660053929140   1478812905  2.321928
                     739747367187387064    1474567514  1.000000

[39106 rows x 2 columns]

In [25]:
interactions_grouped['tsmax'].mean().round(2)
#1470605340.04

1470605340.04

Разумеется, для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.

6. Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Значение отсечки включите в тестовую выборку. Сколько объектов попало в обучающую выборку?

In [34]:
interactions_grouped = interactions_grouped.reset_index()

In [35]:
interactions_grouped

,personId,contentId,tsmax,wsum
0,-1007001694607905623,-5065077552540450930,1470395911,1.000000
1,-1007001694607905623,-6623581327558800021,1487240080,1.000000
2,-1007001694607905623,-793729620925729327,1472834892,1.000000
3,-1007001694607905623,1469580151036142903,1487240062,1.000000
4,-1007001694607905623,7270966256391553686,1485994342,1.584963
...,...,...,...,...
39101,998688566268269815,-401664538366009049,1474567449,1.000000
39102,998688566268269815,3456674717452933449,1478802088,2.584963
39103,998688566268269815,6881796783400625893,1474567675,1.000000
39104,998688566268269815,7174452660053929140,1478812905,2.321928


In [36]:
interactions_train_df = interactions_grouped[interactions_grouped['tsmax']<=1475519545]
interactions_test_df = interactions_grouped[interactions_grouped['tsmax']>1475519545]

In [129]:
interactions_train_df.shape[0]
#29325

29325

In [130]:
final_df = (
    interactions_train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    interactions_test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

7. Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [131]:
content = interactions_train_df.groupby('contentId')['wsum'].agg(['sum']).sort_values(by='sum',ascending=False)
content.iloc[0]
#-6783772548752091658

sum    231.177195
Name: -6783772548752091658, dtype: float64

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

8. Постройте систему рекомендаций. Оцените качество с помощью `precision@10` для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

Для вычисления `precision@10` воспользуйтесь следующей функцией:
```
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
```

Итоговый результат округлите до трёх знаков после точки-разделителя.

*В этом задании необходимо использовать данные, полученные в задании 6.6 (для которых приведён скриншот выше).*

In [139]:
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean().round(3)

In [133]:
def recomendation(row):
    true_train = list(set(row))
    
    recomendation_list = []
    all_recomendations = list(content.reset_index()['contentId'])
    
    for i in list(range(len(all_recomendations))):
        if len(recomendation_list)<10:
            if all_recomendations[i] not in true_train:
                recomendation_list.append(all_recomendations[i])
        else: return recomendation_list

In [134]:
final_df['recomendations'] = final_df['true_train'].apply(lambda x: recomendation(x))

In [140]:
precision('recomendations')
#0.006

0.006

In [138]:
final_df

,true_train,true_test,recomendations
personId,,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72...","[-6783772548752091658, -133139342397538859, -8..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -...","[-6783772548752091658, -133139342397538859, -8..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -...","[-6783772548752091658, -8208801367848627943, 8..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -...","[-133139342397538859, -8208801367848627943, 82..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224],"[-6783772548752091658, -8208801367848627943, 8..."
...,...,...,...
953707509720613429,"[-1068603220639552685, -2358756719610361882, -...","[-2402288292108892893, -5813211845057621660, -...","[-133139342397538859, -8208801367848627943, 82..."
983095443598229476,"[-133139342397538859, -8742648016180281673]","[-14569272361926584, -1572252285162838958, -18...","[-6783772548752091658, -8208801367848627943, 8..."
989049974880576288,"[-133139342397538859, -2038869595290705317, -2...","[-6289909056857931861, -7047448754687279385, -...","[-6783772548752091658, -8208801367848627943, 8..."


Качество получилось не очень высоким, но ведь и рекомендации у нас были неперсонализированными.

**Примечание**. Стоит отметить, что качество РС оценивается не так, как в задачах классификации: показателей выше 0.5 добиться практически невозможно, и даже результат 0.1–0.2 — индикатор высокого качества.

Уже в следующем модуле вы сможете улучшить качество, изучив более продвинутые алгоритмы. Однако уже сейчас у вас получилось построить простейшую систему рекомендаций — это большое достижение!